In [7]:
import numpy as np

import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
from LayerChain import best_layer_chains
from Cliffordize import bricklayer_circ, Cliffordize
from qiskit_ibm_runtime import EstimatorV2
import time


#from qiskit.quantum_info import Clifford, Pauli

In [8]:
nqubits=20
nlayers=10

ncliffordizations=29
nstablizers=30


'''
backend_name = "ibm_peekskill"
instance="grants/qciss/internal"
'''
backend_name = "ibm_fez"
instance="grants/qciss/production"


service = QiskitRuntimeService(instance=instance)
# specify backend and two-qubit gate to be layered
backend = service.backend(backend_name)
twoq_gate = "ecr"
print(f"Device {backend.name} Loaded with {backend.num_qubits} qubits")
print(f"Two Qubit Gate: {twoq_gate}")


qubits_log = [a for a in range(nqubits)]
layer_chains_log = [[(a,b) for a, b in zip(qubits_log[0::2],qubits_log[1::2])],
                    [(a,b) for a, b in zip(qubits_log[1::2],qubits_log[2::2])]]
qubits,layer_chains = best_layer_chains(backend, nqubits,nlayers*int(nqubits/2))


Device ibm_fez Loaded with 156 qubits
Two Qubit Gate: ecr
meas
0.5688679004811545
no meas
0.731843154204214


In [3]:
'''

from qiskit_aer import AerSimulator

backend = AerSimulator(method='stabilizer')

nqubits=5
qubits= [a for a in range(nqubits)]
'''

"\n\nfrom qiskit_aer import AerSimulator\n\nbackend = AerSimulator(method='stabilizer')\n\nnqubits=5\nqubits= [a for a in range(nqubits)]\n"

In [9]:
start = time.time()

target_circ = bricklayer_circ(nqubits,nlayers,gate = 'cz')
outputs = Cliffordize(target_circ,ncliffordizations,nstablizers, backend,gate = 'cz', initial_layout=qubits)
pubs = [val for val in outputs.values()]

end = time.time()
print(end-start)

302.3774757385254


In [10]:


estimator = EstimatorV2(backend)
estimator.options.resilience_level =0
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.resilience.measure_mitigation = False
jobs = estimator.run(pubs)
job_id_01 = jobs.job_id()


In [12]:
jobs = service.job(job_id_01)

means = [np.mean(res.data.evs) for res in jobs.result()]
print(np.array(means)/means[0])

[1.         0.56255952 0.55742779 0.5481166  0.55401545 0.55999365
 0.55787747 0.55216379 0.56829965 0.56491377 0.5617395  0.54478362
 0.56269178 0.56017882 0.55808909 0.56348535 0.56033753 0.54994181
 0.56792932 0.57837795 0.56049624 0.56499312 0.55263993 0.56015236
 0.55882975 0.56869643 0.5700455  0.55102635 0.5629034  0.56337954]


In [13]:
errs = 1-np.array(means[1:])/means[0]
print(np.mean(errs))
print(np.std(errs))

0.4397901699874854
0.007107705199954822


In [15]:
1-(1-np.mean(errs))**(1/(nlayers*int(nqubits/2)))

0.0057776833082650025

In [17]:
(nlayers*int(nqubits/2))

100

In [1]:
1-(1-0.4397901699874854)**(1/(5*(10+9)))

0.006080845512795752